# Bolivian League Betting Odds
## 1. Data Cleaning & Feature Engineering

In [5]:
# Importing necessary packages
import pandas as pd
import numpy as np
import matplotlib.style as style
import matplotlib.pyplot as plt
import seaborn as sns
import string
import matplotlib.ticker as ticker
import re
from collections import Counter
style.use('fivethirtyeight')



from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [12]:
# Reading and appending datasets
odds_16_08 = pd.read_csv('odds_data/raw/odds_portal_2016_2008.csv', parse_dates= ['date'] )
odds_17 = pd.read_csv('odds_data/raw/odds_portal_2017.csv', parse_dates= ['date'])
odds_18 = pd.read_csv('odds_data/raw/odds_portal_2018.csv', parse_dates= ['date'])
odds_19 = pd.read_csv('odds_data/raw/odds_portal_2019.csv', parse_dates= ['date'])

odds = odds_16_08.append([odds_17, odds_18, odds_19])



In [13]:
odds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24758 entries, 0 to 3739
Data columns (total 12 columns):
web-scraper-order        24758 non-null object
web-scraper-start-url    24758 non-null object
match                    24758 non-null object
match-href               24758 non-null object
Bookmakers               21802 non-null object
Home Odds                21802 non-null float64
Draw Odds                21802 non-null float64
Away Odds                21802 non-null float64
Payout                   21802 non-null object
results                  24758 non-null object
date                     24758 non-null datetime64[ns]
time                     24758 non-null object
dtypes: datetime64[ns](1), float64(3), object(8)
memory usage: 2.5+ MB


### Cleaning the dataframe


In [14]:
# Making column names easier to work with
odds.columns = [c.lower().replace("-", "_") for c in [i.replace(" ", "-") for i in odds.columns]]
odds.columns

# Payout column
odds['payout'] = round(pd.to_numeric(odds.payout.str.replace('%',''), errors='coerce')/100,2)

# Dropping NA's
odds.dropna(subset=['bookmakers','payout'],axis=0, inplace=True)

# CREATING NEW COLUMNS

# Season Year
odds['season_year'] = odds.date.dt.year
# Teams
odds['home_team'] = [i[0].strip() for i in odds.match.str.split("-")]
odds['away_team'] = [i[1].strip() for i in odds.match.str.split("-")]

# Final Results
odds['final_result'] = [c[0:3] for c in [i.replace('Final result ', '') for i in odds.results]]
odds['home_goals'] = [i[0] for i in odds.final_result.str.split(':')]
odds['away_goals'] = [i[-1] for i in odds.final_result.str.split(':')]

# Half times results
odds['halftime'] = odds.results.str.extract(r"\((.*?)\)", expand=False).str.split(',')

odds.dropna(subset=['halftime'],inplace=True) # dropping rows that don't have half time info

odds['first_half_home'] = [c[0] for c in [i[0] for i in odds.halftime]]
odds['first_half_away'] = [c[2] for c in [i[0] for i in odds.halftime]]
odds['second_half_home'] = [c[0] for c in [i[1].strip() for i in odds.halftime]]
odds['second_half_away'] = [c[2] for c in [i[1].strip() for i in odds.halftime]]

# Fixing unique cases in score
odds.home_goals[odds.home_goals=='Wil'] = 7
odds.away_goals[odds.away_goals=='Wil'] = 0


# Dropping columns
odds.drop(['halftime','web_scraper_start_url',
           'results', 'web_scraper_order'], axis=1, inplace=True)


Index(['web_scraper_order', 'web_scraper_start_url', 'match', 'match_href',
       'bookmakers', 'home_odds', 'draw_odds', 'away_odds', 'payout',
       'results', 'date', 'time'],
      dtype='object')

/Users/miketondu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/miketondu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Basic Feature Creation
Create simple features based on the goals scored throughout each individual match

In [ ]:
# Creating new features
# Transforming values into numeric type
for col in odds.columns[odds.columns!='date']:
    odds[col] = pd.to_numeric(odds[col], errors='ignore')

# Total number of goals
odds['total_goals'] = odds.home_goals + odds.away_goals

# First half goals
odds['total_first_half'] = odds.first_half_home + odds.first_half_away

# Second half goals
odds['total_second_half'] = odds.second_half_home + odds.second_half_away

# Penalties
#odds.results[odds.results.str.contains('penalties')]

# win_home_or_away (1 for home win, 2 for away win, 0 for draw)
def win_column(dataframe):
    if dataframe['home_goals'] > dataframe['away_goals']:
        return 1
    if dataframe['home_goals'] < dataframe['away_goals']:
        return 2
    else:
        return 0    
odds['win_home_or_away'] = odds.apply(lambda x: win_column(x) , axis=1)

# winning team
odds['winner'] = ''
for i in range(len(odds)):
    if odds['win_home_or_away'].values[i] ==1:
        odds['winner'].values[i] = odds['home_team'].values[i]
    elif odds['win_home_or_away'].values[i] ==2:
        odds['winner'].values[i] = odds['away_team'].values[i]
    else:
        odds['winner'].values[i] = 'Draw'
        
        
# Creating 4 columns for home win/loss, and away win/loss
odds['home_win']=0
odds['away_win']=0
odds['home_loss']=0
odds['away_loss']=0
for i in range(len(odds)):
    if (odds['win_home_or_away'].values[i] == 1):
        odds['home_win'].values[i]=1
        odds['away_loss'].values[i]=1
    if(odds['win_home_or_away'].values[i] == 2):
        odds['away_win'].values[i]=1
        odds['home_loss'].values[i]=1


# Implied odds ------> Could use this value to compare against model output
odds['implied_home_odds'] = round(1/odds.home_odds,3)
odds['implied_draw_odds'] = round(1/odds.draw_odds,3)
odds['implied_away_odds'] = round(1/odds.away_odds,3)

# Create 'upsets' variable ---> 0 for no upset, 1 for upset
# Upset defined as home team losing against away team with lower odds of winning

def upset_column(dataframe):
    if (dataframe['implied_home_odds']
        > dataframe['implied_away_odds']) & (dataframe['home_goals'] <
                                             dataframe['away_goals']):
        return 1
    else:
        return 0 
odds['upset'] = odds.apply(lambda x: upset_column(x),axis=1) 


        

# saving df to .csv for future notebooks
odds.to_csv('/Users/miketondu/Dropbox/Data Science/Sharpest Minds/odds_data/bolivian_football_odds_clean.csv', index=False)


In [ ]:
odds.date.max()

### Data Cleaning Handicap Data

In [12]:
# Loading Handicap data
hdc = pd.read_csv('/Users/miketondu/Downloads/odds_handicap (2).csv')

hdc.head()

FileNotFoundError: File b'/Users/miketondu/Downloads/odds_handicap (2).csv' does not exist

In [13]:
len(hdc)

NameError: name 'hdc' is not defined